This file is for setting up the individual Rook classes so that they work seemlessly together

In [31]:
class Board:
    
    def __init__(self, initial_state = None, bounds = None):
        """
        The board constructor initializes the current move set and game state.

        Args:
            initial_state : The initial game state
            bounds : The bounds of the board, if applicable (size, minimum, etc.)
        """
        self.initial_state = initial_state
        self.state = initial_state
        self.bounds = bounds
        self.data = {}
    
    def reset():
        """
        Resets the board to its original state.
        """
        self.state = self.initial_state
        self.data = {}

In [32]:
class Player:
    
    def __init__(self, name, strategy):
        """
        The player constructor.

        Args:
            name: The player's label
            strategy: A strategy function used by the player
        """
        self.name = name
        self.strategy = strategy

    def move(self, board):
        """
        This function takes in a board and then allows the player to make
        a choice on the next move.

        Args:
            board : The board object for the current game

        Return:
            The move made.
        """
        return self.strategy.move(board)

In [33]:
class Referee:
    
    def __init__(self, board, rules):
        """
        This class models a referee for a game.
        
        Args:
            board : The board of the game being played
            rules : The ruleset of the current game
        """
        self.board = board
        self.rules = rules

    def update_board(self, board, player, move):
        """
        This method will handle updating the game board.

        Args:
            board : The game board
            move : The move made
        """
        self.rules.update_board(board, player, move)

    def ask_for_move(self, player, board):
        """
        Asks the current player for a proposed move.

        Args:
            player : Player whose turn it is
            board : The game board

        Return:
            The players move, or None if the move failed.
        """

        # Ask for a move from the player
        proposed_move = player.move(board)

        # Check if the player has given up or if the move is illegal
        if not self.rules.is_legal(self.board, proposed_move):
            proposed_move = None
        
        # Otherwise, the move was valid, so return the proposed move
        return proposed_move
        
    def is_game_over(self, board):
        """
        Checks to see if the game is over.
        
        Args:
            board : The game board
        
        Return:
            True if the game is over, else False.
        """
        return self.rules.is_game_over(board)
    
    def declare_winner(self, board, player):
        """
        Declares a winner to the game.
        
        Args:
            board : The game board
            player : The current player making the move
        """
        board.data["winner"] = player.name

In [77]:
class Game:
    
    def __init__(self, referee, board, players):
        """
        Creates a new game with the given referee, board, and players.

        Args:
            referee : The current game referee
            board : The game board
            players : The list of players
        """
        self.referee = referee
        self.board = board
        self.player = players[0]
        self.opponent = players[1]
        

    def play(self):
        """
        Plays a game on the current board, moderated by the referee, with the two players.

        Return:
            The game board being played on
        """
        count = 0
        winner = None
        while winner is None:   
            print("State: " + str(self.board.state))

            # Request a move from the current player
            move = self.referee.ask_for_move(self.player, self.board)
            print(self.player.name + ": " + str(move))
            
            # If the move is NOT valid, exit the loop
            if move is None:
                winner = self.opponent
                print("LOSER")

            # Tell the referee to update the board with the (valid) move
            else:
                self.referee.update_board(self.board, self.player, move)
            
            # Check if the game has been won
            if self.referee.is_game_over(self.board):
                winner = self.player
                print("WINNER FOUND")
            
            # Next turn
            self.player, self.opponent = self.opponent, self.player
            
            count += 1
            if count == 50:
                print("INFINITE")
                break
            
        # Declare a winner to the game
        self.referee.declare_winner(self.board, winner)
        return self.board

In [11]:
class ToothpickRuleset:
    
    def __init__(self, name, initial_state = 10):
        """
        Rulset constructor.
        
        Args:
            name : The name of the ruleset
            initial_state : The number of toothpicks to start with
        """
        self.name = name
        self.initial_state = initial_state
        self.bounds = None
    
    def is_legal(self, board, proposed_move):
        """
        Determines if the move proposed is legal.
        
        Args:
            board : The board being played on
            proposed_move : The move being proposed
        
        Return:
            True if the move greater than 0 and less than the number of toothpicks left or 2, whichever is smaller.
        """
        return proposed_move > 0 and proposed_move <= min(2, board.state)
    
    def is_game_over(self, board):
        """
        Determines if the game is over.
        
        Args:
            board : The board being played on
        
        Return:
            True if no more toothpicks left, else False.
        """
        return board.state == 0
    
    def update_board(self, board, player, move):
        """
        Updates the number of toothpicks left on the table.
        
        Args:
            board : The board being played on
            player : The player who made the move
            move : The number of toothpicks taken
        """
        board.data[board.state] = (player.name, move)
        board.state -= move

In [12]:
import random
class ToothpickStrategy:
    
    def __init__(self, name, data = None):
        """
        Strategy constructor.
        
        Args:
            name : The name of the strategy
            data : Data to read from, if applicable
        """
        self.name = name
    
    def move(self, board):
        """
        Make a move based on the strategy.
        
        Args:
            board : The board being played on
            
        Return:
            A proposed move.
        """
        return random.randrange(1, 3)

In [47]:
def simulate(ruleset, strategy):
    p1 = Player("Player1", strategy)
    p2 = Player("Player2", strategy)
    
    board = Board(ruleset.initial_state, ruleset.bounds)
    
    ref = Referee(board, ruleset)
    
    game = Game(ref, board, [p1, p2])
    b = game.play()
    
    return b.data

In [ ]:
toothpick_ruleset = ToothpickRuleset("Basic Toothpick Takeaway Ruleset", 10)
toothpick_strat = ToothpickStrategy("Random guess strategy for ToothpickTakeaway")
simulate(toothpick_ruleset, toothpick_strat)

In [49]:
class RookRuleset:
    def __init__(self, name, initial_state = {"D": 0, "R": 0}, bounds = {"D": 9, "R": 9}):
        """
        Rulset constructor.
        
        Args:
            name : The name of the ruleset
            initial_state : The initial state of the board
            bounds : The bounds of the board
        """
        self.name = name
        self.initial_state = initial_state
        self.bounds = bounds
    
    def is_legal(self, board, proposed_move):
        """
        Determines if the move proposed is legal.
        
        Args:
            board : The board being played on
            proposed_move : The move being proposed
        
        Return:
            True if the move's direction is Down or Right AND it is within bounds of the board
        """
        legal = True
        if proposed_move["direction"] == "D":
            legal = board.state["D"] + proposed_move["tiles"] <= board.bounds["D"]
        elif proposed_move["direction"] == "R":
            legal = board.state["R"] + proposed_move["tiles"] <= board.bounds["R"]
        return legal

    def is_game_over(self, board):
        """
        Updates the rook's position on the board.
        
        Args:
            board : The board being played on
            
        Return:
            True if the current board state is its bounds (rook is at the end of the board).
        """
        return board.state == board.bounds
    
    def update_board(self, board, player, move):
        """
        Updates the rook's position on the board by applying the move's direction and number of tiles.
        
        Args:
            board : The board being played on
            player : The player who made the move
            move : The move being made
        """
        #if move["direction"] == "D":
        #    bord.state["D"] += move["tiles"]
        #elif move["direction"] == "R":
        #    bord.state["R"] += move["tiles"]
        state = "{},{}".format(board.state["R"], board.state["D"])
        board.data[state] = (player.name, move)

        board.state[move["direction"]] += move["tiles"]

In [50]:
import random
class RookStrategy:
    
    def __init__(self, name, data = None):
        """
        Strategy constructor.
        
        Args:
            name : The name of the strategy
            data : Data to read from, if applicable
        """
        self.name = name
    
    def move(self, board):
        """
        Make a move based on the strategy.
        
        Args:
            board : The board being played on
            
        Return:
            A proposed move.
        """
        direction = random.choice(["D", "R"])
        #move = random.randrange(0, 11)
        
        # Smart Move
        max_tiles = board.bounds[direction] - board.state[direction]
        move = random.randrange(0, max_tiles + 1)

        return {"direction": direction, "tiles": move}

In [78]:
rook_ruleset = RookRuleset("Basic Unbalanced Rook Ruleset", initial_state = {"D": 0, "R": 0}, bounds = {"D": 9, "R": 9})
rook_strat = RookStrategy("Random guess strategy for Unbalanced Rook")
simulate(rook_ruleset, rook_strat)

State: {'D': 0, 'R': 0}
Player1: {'direction': 'D', 'tiles': 5}
State: {'D': 5, 'R': 0}
Player2: {'direction': 'R', 'tiles': 9}
State: {'D': 5, 'R': 9}
Player1: {'direction': 'D', 'tiles': 2}
State: {'D': 7, 'R': 9}
Player2: {'direction': 'D', 'tiles': 1}
State: {'D': 8, 'R': 9}
Player1: {'direction': 'D', 'tiles': 1}
WINNER FOUND


{'0,0': ('Player1', {'direction': 'D', 'tiles': 5}),
 '0,5': ('Player2', {'direction': 'R', 'tiles': 9}),
 '9,5': ('Player1', {'direction': 'D', 'tiles': 2}),
 '9,7': ('Player2', {'direction': 'D', 'tiles': 1}),
 '9,8': ('Player1', {'direction': 'D', 'tiles': 1}),
 'winner': 'Player1'}